<a href="https://colab.research.google.com/github/SahajTC/Augmented-Financial-Intelligence/blob/main/LSTMipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import os
import pandas as pd
import numpy as np
import math
import datetime as dt

In [23]:
pip install yfinance prophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score,mean_absolute_percentage_error 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM

In [26]:
import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [27]:
import yfinance as yf
btc=yf.download(tickers='BTC-USD', start='2020-01-01', end='2022-11-30', interval = '1d')#bitcoin
eth=yf.download(tickers='ETH-USD', start='2020-01-01', end='2022-11-30', interval = '1d')#etherum
bnb=yf.download(tickers='BNB-USD', start='2020-01-01', end='2022-11-30', interval = '1d')#binance

aapl = yf.download(tickers='AAPL', start='2020-01-01', end='2022-11-30', interval = '1d')#apple
tsla=yf.download(tickers='TSLA', start='2020-01-01', end='2022-11-30', interval = '1d')#tesla
googl=yf.download(tickers='GOOGL', start='2020-01-01', end='2022-11-30', interval = '1d')#google

doge=yf.download(tickers='DOGE-USD', start='2020-01-01', end='2022-11-30', interval = '1d')#dogecoin
dot=yf.download(tickers='DOT-USD', start='2020-01-01', end='2022-11-30', interval = '1d')#polkadot
atom=yf.download(tickers='ATOM-USD', start='2020-01-01', end='2022-11-30', interval = '1d')#cosmos

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [28]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]    
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [38]:
def Model_fit(bth):
 bth.reset_index(inplace=True)
 bth['Date'] = pd.to_datetime(bth['Date'],format='%Y-%m-%d')
 closebth = bth[['Date','Close']]
 closebth = closebth[closebth['Date'] > '2020-01-01']

 del closebth['Date']
 scaler=MinMaxScaler(feature_range=(0,1))
 closebth=scaler.fit_transform(np.array(closebth).reshape(-1,1))

 training_size=int(len(closebth)*0.80)
 test_size=len(closebth)-training_size
 train_data,test_data=closebth[0:training_size,:],closebth[training_size:len(closebth),:1]

 time_step = 15
 X_train, y_train = create_dataset(train_data, time_step)
 X_test, y_test = create_dataset(test_data, time_step)
 X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
 X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

 model=Sequential()
 model.add(LSTM(10,input_shape=(None,1),activation="relu"))
 model.add(Dense(1))
 model.compile(loss="mean_squared_error",optimizer="adam")

 history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=32,verbose=1)

 train_predict=model.predict(X_train)
 test_predict=model.predict(X_test)

 train_predict = scaler.inverse_transform(train_predict)
 test_predict = scaler.inverse_transform(test_predict)
 original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
 original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

 print("Train data explained MAPE score:", 
      mean_absolute_percentage_error(original_ytrain, train_predict))
 print("Test data explained MAPE score:", 
      mean_absolute_percentage_error(original_ytest, test_predict))

In [30]:
Model_fit(btc)

Epoch 1/200
26/26 [==============================] - 2s 28ms/step - loss: 0.2463 - val_loss: 0.0455
Epoch 2/200
26/26 [==============================] - 0s 11ms/step - loss: 0.1874 - val_loss: 0.0194
Epoch 3/200
26/26 [==============================] - 0s 13ms/step - loss: 0.1076 - val_loss: 0.0026
Epoch 4/200
26/26 [==============================] - 0s 11ms/step - loss: 0.0161 - val_loss: 0.0013
Epoch 5/200
26/26 [==============================] - 0s 11ms/step - loss: 0.0023 - val_loss: 6.3903e-04
Epoch 6/200
26/26 [==============================] - 0s 10ms/step - loss: 0.0019 - val_loss: 9.3043e-04
Epoch 7/200
26/26 [==============================] - 0s 11ms/step - loss: 0.0018 - val_loss: 9.0977e-04
Epoch 8/200
26/26 [==============================] - 0s 9ms/step - loss: 0.0017 - val_loss: 9.2113e-04
Epoch 9/200
26/26 [==============================] - 0s 10ms/step - loss: 0.0016 - val_loss: 0.0010
Epoch 10/200
26/26 [==============================] - 0s 12ms/step - loss: 0.0016 - v

In [31]:
Model_fit(eth)

Epoch 1/200
26/26 [==============================] - 2s 24ms/step - loss: 0.2694 - val_loss: 0.0718
Epoch 2/200
26/26 [==============================] - 0s 10ms/step - loss: 0.1914 - val_loss: 0.0415
Epoch 3/200
26/26 [==============================] - 0s 9ms/step - loss: 0.1367 - val_loss: 0.0222
Epoch 4/200
26/26 [==============================] - 0s 9ms/step - loss: 0.0846 - val_loss: 0.0075
Epoch 5/200
26/26 [==============================] - 0s 11ms/step - loss: 0.0261 - val_loss: 0.0010
Epoch 6/200
26/26 [==============================] - 0s 13ms/step - loss: 0.0035 - val_loss: 0.0015
Epoch 7/200
26/26 [==============================] - 0s 15ms/step - loss: 0.0025 - val_loss: 0.0022
Epoch 8/200
26/26 [==============================] - 0s 13ms/step - loss: 0.0024 - val_loss: 0.0025
Epoch 9/200
26/26 [==============================] - 0s 14ms/step - loss: 0.0023 - val_loss: 0.0029
Epoch 10/200
26/26 [==============================] - 0s 16ms/step - loss: 0.0023 - val_loss: 0.0026
E

In [36]:
Model_fit(bnb)

Epoch 1/200
26/26 [==============================] - 2s 25ms/step - loss: 0.0542 - val_loss: 0.0172
Epoch 2/200
26/26 [==============================] - 0s 12ms/step - loss: 0.0108 - val_loss: 0.0078
Epoch 3/200
26/26 [==============================] - 0s 12ms/step - loss: 0.0071 - val_loss: 0.0046
Epoch 4/200
26/26 [==============================] - 0s 15ms/step - loss: 0.0051 - val_loss: 0.0035
Epoch 5/200
26/26 [==============================] - 0s 13ms/step - loss: 0.0038 - val_loss: 0.0028
Epoch 6/200
26/26 [==============================] - 0s 13ms/step - loss: 0.0030 - val_loss: 0.0015
Epoch 7/200
26/26 [==============================] - 0s 14ms/step - loss: 0.0024 - val_loss: 8.5276e-04
Epoch 8/200
26/26 [==============================] - 0s 12ms/step - loss: 0.0020 - val_loss: 7.6640e-04
Epoch 9/200
26/26 [==============================] - 0s 12ms/step - loss: 0.0017 - val_loss: 4.6992e-04
Epoch 10/200
26/26 [==============================] - 0s 13ms/step - loss: 0.0016 - val_

In [39]:
Model_fit(doge)

Epoch 1/200
27/27 [==============================] - 2s 19ms/step - loss: 0.0343 - val_loss: 0.0013
Epoch 2/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0133 - val_loss: 2.3841e-04
Epoch 3/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0031 - val_loss: 2.8746e-04
Epoch 4/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0021 - val_loss: 2.4436e-04
Epoch 5/200
27/27 [==============================] - 0s 10ms/step - loss: 0.0019 - val_loss: 2.4681e-04
Epoch 6/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0019 - val_loss: 2.5113e-04
Epoch 7/200
27/27 [==============================] - 0s 10ms/step - loss: 0.0019 - val_loss: 2.7160e-04
Epoch 8/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0018 - val_loss: 2.1882e-04
Epoch 9/200
27/27 [==============================] - 0s 10ms/step - loss: 0.0017 - val_loss: 2.4376e-04
Epoch 10/200
27/27 [==============================] - 0s 11ms/step - loss

In [34]:
Model_fit(dot)

Epoch 1/200
21/21 [==============================] - 2s 26ms/step - loss: 0.0680 - val_loss: 4.5433e-04
Epoch 2/200
21/21 [==============================] - 0s 12ms/step - loss: 0.0266 - val_loss: 0.0086
Epoch 3/200
21/21 [==============================] - 0s 12ms/step - loss: 0.0088 - val_loss: 0.0128
Epoch 4/200
21/21 [==============================] - 0s 11ms/step - loss: 0.0069 - val_loss: 0.0072
Epoch 5/200
21/21 [==============================] - 0s 12ms/step - loss: 0.0055 - val_loss: 0.0049
Epoch 6/200
21/21 [==============================] - 0s 11ms/step - loss: 0.0048 - val_loss: 0.0028
Epoch 7/200
21/21 [==============================] - 0s 11ms/step - loss: 0.0044 - val_loss: 0.0022
Epoch 8/200
21/21 [==============================] - 0s 11ms/step - loss: 0.0042 - val_loss: 0.0012
Epoch 9/200
21/21 [==============================] - 0s 12ms/step - loss: 0.0041 - val_loss: 0.0014
Epoch 10/200
21/21 [==============================] - 0s 12ms/step - loss: 0.0039 - val_loss: 0.

In [40]:
Model_fit(atom)

Epoch 1/200
27/27 [==============================] - 2s 19ms/step - loss: 0.1233 - val_loss: 0.0195
Epoch 2/200
27/27 [==============================] - 0s 10ms/step - loss: 0.0700 - val_loss: 0.0029
Epoch 3/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0188 - val_loss: 0.0024
Epoch 4/200
27/27 [==============================] - 0s 11ms/step - loss: 0.0044 - val_loss: 6.7749e-04
Epoch 5/200
27/27 [==============================] - 0s 11ms/step - loss: 0.0034 - val_loss: 6.9929e-04
Epoch 6/200
27/27 [==============================] - 0s 10ms/step - loss: 0.0030 - val_loss: 5.6439e-04
Epoch 7/200
27/27 [==============================] - 0s 10ms/step - loss: 0.0028 - val_loss: 5.0717e-04
Epoch 8/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0027 - val_loss: 5.1385e-04
Epoch 9/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0025 - val_loss: 5.0438e-04
Epoch 10/200
27/27 [==============================] - 0s 11ms/step - loss: 0.00